<a href="https://colab.research.google.com/github/gayoooon1/2021-RecSys/blob/main/Non_Personalized_Recommendation_System_using_1M_MovieLens_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Non-Personalized Recommendation System using 1M MovieLens Data

## 목표
1. 앞으로 사용할 데이터의 형태에 익숙해지고, 데이터 다루는 법을 터득하기
2. 가장 간단한 추천시스템 구현해보기

## 방법
0. 추천모델에서 리뷰점수는 binarize 하기 1~5 scale 을 다 1로 해주고, 안본 영화는 0으로 처리하기
1. 각 이용자마다 5%-10%를 testing 으로 남겨두고 나머지 90-95%로  추천시스템을 만들기
2. 추천시스템을 만들고 검증하기 (검증 방법 : 5~10%의 데이터를 활용하여 검증.  recall@K, hitrate@K 등을 이용하여 할 수 있음)
3. 제일 많이 보는 영화를 추천 = 1등으로 많이 본 영화, 2등으로 많이 본 영화, 3등으로 많이 본 영화,   ...    K(50으로 설정) 등으로 많이 본 영화 인기있는 영화 추천/ 훈련데이터에서 어떤 영화를 가장 많이 봤는지(1,2,...k등까지) k등 순위로 영화를 추천한다고 가정할 때 이것을 검증. 
테스트 데이터 안에서 누가 어떤 것을 봤는지 실제로 본 영화가 랭킹에 몇 순위에 있는지 sum으로 counting, 유저와 상관없이 추천! 유저가 과거에 어떤 영화봤는지는 상관없고 인기있는 영화를 추처하면 됨. 인기있는 영화는 training set에서 찾으면 됨 1들만 골라서 안 본 영화는 고려하지 않고 본 영화들만 몇 순위로 예측을 하는지 간단히 검증 1로 예측하면 몇 순위 영화다. test 셋 안에서 몇 순위로 예측이 될거다 측정하기/ non-personal 안본 영화는 test 데이터 안에서 생각 안해도 된다 본 영화만 몇 순위로 예측

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/DEEPLEARNING/ml-1m

/content/drive/MyDrive/DEEPLEARNING/ml-1m


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
movies = pd.read_csv('movies.dat', header=None, sep='::', names=["MovieID", "Title", "Genres"], encoding='latin-1', engine='python')
ratings = pd.read_csv('ratings.dat', header=None, sep='::', names=['UserID','MovieID','Rating','Timestamp'], encoding='latin-1', engine='python')

In [ ]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings = ratings[['UserID','MovieID','Rating']]
ratings_matrix = ratings.pivot_table('Rating', index='UserID', columns='MovieID')
ratings_matrix.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,4.0,NaN,3.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# title 칼럼을 얻기 위해 movies와 조인
rating_movies = pd.merge(ratings,movies, on='MovieID')

# columns='title'로 title 칼럼으로 피벗 수행
ratings_matrix = rating_movies.pivot_table('Rating', index='UserID', columns='Title')

In [ ]:
ratings_matrix.head()

Title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",187 (1997),2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010 (1984),24 7: Twenty Four Seven (1997),24-hour Woman (1998),28 Days (2000),3 Ninjas: High Noon On Mega Mountain (1998),3 Strikes (2000),"301, 302 (1995)","39 Steps, The (1935)","400 Blows, The (Les Quatre cents coups) (1959)",42 Up (1998),52 Pick-Up (1986),54 (1998),"7th Voyage of Sinbad, The (1958)",8 1/2 (1963),8 1/2 Women (1999),8 Heads in a Duffel Bag (1997),8 Seconds (1994),8MM (1999),A Chef in Love (1996),Abbott and Costello Meet Frankenstein (1948),"Abominable Snowman, The (1957)",About Adam (2000),About Last Night... (1986),...,Wonderland (1997),Wonderland (1999),Woo (1998),"Wood, The (1999)","Wooden Man's Bride, The (Wu Kui) (1994)",Working Girl (1988),"World Is Not Enough, The (1999)","World of Apu, The (Apur Sansar) (1959)","Wrong Man, The (1956)","Wrong Trousers, The (1993)",Wrongfully Accused (1998),Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X: The Unknown (1956),Xiu Xiu: The Sent-Down Girl (Tian yu) (1998),Yankee Zulu (1994),"Yards, The (1999)","Year My Voice Broke, The (1987)",Year of Living Dangerously (1982),Year of the Horse (1997),Yellow Submarine (1968),Yojimbo (1961),You Can't Take It With You (1938),You So Crazy (1994),You've Got Mail (1998),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# binarize
for a in ratings_matrix.columns:
  ratings_matrix[a] = ratings_matrix[a].map({1:1, 2:1, 3:1, 4:1, 5:1})

In [ ]:
ratings_matrix.fillna(0, inplace=True)

In [ ]:
ratings_matrix.head()

Title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",187 (1997),2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010 (1984),24 7: Twenty Four Seven (1997),24-hour Woman (1998),28 Days (2000),3 Ninjas: High Noon On Mega Mountain (1998),3 Strikes (2000),"301, 302 (1995)","39 Steps, The (1935)","400 Blows, The (Les Quatre cents coups) (1959)",42 Up (1998),52 Pick-Up (1986),54 (1998),"7th Voyage of Sinbad, The (1958)",8 1/2 (1963),8 1/2 Women (1999),8 Heads in a Duffel Bag (1997),8 Seconds (1994),8MM (1999),A Chef in Love (1996),Abbott and Costello Meet Frankenstein (1948),"Abominable Snowman, The (1957)",About Adam (2000),About Last Night... (1986),...,Wonderland (1997),Wonderland (1999),Woo (1998),"Wood, The (1999)","Wooden Man's Bride, The (Wu Kui) (1994)",Working Girl (1988),"World Is Not Enough, The (1999)","World of Apu, The (Apur Sansar) (1959)","Wrong Man, The (1956)","Wrong Trousers, The (1993)",Wrongfully Accused (1998),Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X: The Unknown (1956),Xiu Xiu: The Sent-Down Girl (Tian yu) (1998),Yankee Zulu (1994),"Yards, The (1999)","Year My Voice Broke, The (1987)",Year of Living Dangerously (1982),Year of the Horse (1997),Yellow Submarine (1968),Yojimbo (1961),You Can't Take It With You (1938),You So Crazy (1994),You've Got Mail (1998),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ratings_matrix.mean().sort_values(ascending=False)

Title
American Beauty (1999)                                   0.567550
Star Wars: Episode IV - A New Hope (1977)                0.495199
Star Wars: Episode V - The Empire Strikes Back (1980)    0.495033
Star Wars: Episode VI - Return of the Jedi (1983)        0.477318
Jurassic Park (1993)                                     0.442384
                                                           ...   
Anna (1996)                                              0.000166
McCullochs, The (1975)                                   0.000166
Shadows (Cienie) (1988)                                  0.000166
Night Tide (1961)                                        0.000166
Another Man's Poison (1952)                              0.000166
Length: 3706, dtype: float64

In [ ]:
ratings_matrix.sum().sort_values(ascending=False)

Title
American Beauty (1999)                                   3428.0
Star Wars: Episode IV - A New Hope (1977)                2991.0
Star Wars: Episode V - The Empire Strikes Back (1980)    2990.0
Star Wars: Episode VI - Return of the Jedi (1983)        2883.0
Jurassic Park (1993)                                     2672.0
                                                          ...  
Anna (1996)                                                 1.0
McCullochs, The (1975)                                      1.0
Shadows (Cienie) (1988)                                     1.0
Night Tide (1961)                                           1.0
Another Man's Poison (1952)                                 1.0
Length: 3706, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test = train_test_split(ratings_matrix, test_size = 0.05, random_state = 42)

In [ ]:
top_50_rec = X_train.mean().sort_values(ascending=False).head(50).index.tolist()
top_50_rec

['American Beauty (1999)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Saving Private Ryan (1998)',
 'Matrix, The (1999)',
 'Back to the Future (1985)',
 'Silence of the Lambs, The (1991)',
 'Men in Black (1997)',
 'Fargo (1996)',
 'Raiders of the Lost Ark (1981)',
 'Sixth Sense, The (1999)',
 'Braveheart (1995)',
 'Shakespeare in Love (1998)',
 'Princess Bride, The (1987)',
 "Schindler's List (1993)",
 'L.A. Confidential (1997)',
 'Groundhog Day (1993)',
 'E.T. the Extra-Terrestrial (1982)',
 'Being John Malkovich (1999)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Shawshank Redemption, The (1994)',
 'Godfather, The (1972)',
 'Ghostbusters (1984)',
 'Forrest Gump (1994)',
 'Pulp Fiction (1994)',
 'Terminator, The (1984)',
 'Toy Story (1995)',
 'Alien (1979)',
 'Total Recall (1990)',
 'Fugitive, Th

In [ ]:
X_test.index

Int64Index([5530,  711, 4924, 2154, 1273, 2317, 2095, 3128,  743,  297,
            ...
            4344,  372, 5668, 2084, 1074, 3342, 4649,  324, 2944, 4367],
           dtype='int64', name='UserID', length=302)

In [ ]:
from scipy import sparse

In [ ]:
#sparse_matrix = sparse.coo_matrix((X_test, (X_test.index, X_test.columns)))
#print("{}".format(sparse_matrix))

In [ ]:
#X_test.loc[711].index.tolist()

In [ ]:
# 영화 제목이 순위 안에 있으면 순위를 알려주는 함수
def find_rank(movie_name, top_50_rec):
  for tmp in range(len(top_50_rec)):
    if movie_name in top_50_rec[tmp]:
      return tmp+1

In [ ]:
# hitrate 검증, 본 영화 목록과 rank, hit을 알려주는 함수
def hitrate_k(X_test):
  hit_list=[]
  #hit=0
  mean_total=0
  for a in X_test.index:
    print('User :' ,a)
    hit = 0
    for b in top_50_rec:
     if X_test.loc[a][b] == 1:
       hit += 1
       print(b , 'ranked #' , find_rank(b, top_50_rec))
    print('Hit :', hit)
    mean_total += hit/50
  return mean_total

# Hitrate at K
K를 50으로 했을 때,   test data안에 (user, item) pair 가 몇개인지 보고, 이중에 몇개가 ranking 50 안에 들어왔는지 그 비율

## Using for loop

In [ ]:
def hitrate_50(X_test):
  mean_total=0
  for idx in X_test.index:
    hit=0
    pairs=0
    for col in X_test.columns:
     if X_test.loc[idx][col] == 1:
       pairs += 1
       if col in top_50_rec:
         hit += 1  
    mean_total = hit/pairs
    print(idx, hit, pairs,mean_total)

In [ ]:
%%timeit
hitrate_50(X_test)

5530 36 652 0.05521472392638037
711 25 143 0.17482517482517482
4924 11 41 0.2682926829268293
2154 13 45 0.28888888888888886
1273 30 334 0.08982035928143713
2317 12 61 0.19672131147540983
2095 11 75 0.14666666666666667
3128 35 317 0.11041009463722397
743 10 149 0.06711409395973154
297 26 89 0.29213483146067415
3695 38 302 0.12582781456953643
1143 12 53 0.22641509433962265
409 37 297 0.12457912457912458
5286 17 182 0.09340659340659341
3950 6 70 0.08571428571428572
1621 2 22 0.09090909090909091
2054 37 335 0.11044776119402985
5594 18 130 0.13846153846153847
5759 43 576 0.07465277777777778
866 8 36 0.2222222222222222
806 8 66 0.12121212121212122
656 27 177 0.15254237288135594
4818 16 50 0.32
2516 7 22 0.3181818181818182
5671 22 109 0.2018348623853211
24 26 136 0.19117647058823528
1422 46 728 0.06318681318681318
5655 20 117 0.17094017094017094
5256 46 825 0.055757575757575756
2437 15 42 0.35714285714285715
4161 22 459 0.04793028322440087
1631 17 260 0.06538461538461539
2518 7 42 0.166666666

KeyboardInterrupt: ignored

In [ ]:
hitrate_50(X_test[:3])

## Not using for loop

In [ ]:
def hitrate_50_faster(X_test):
  X_test['Hit'] = 0
  mean_total = 0
  for col in X_test.columns:
    X_test.loc[(X_test[col] == 1) & (col in top_50_rec), 'Hit'] += 1
  mean_total = X_test['Hit']/X_test.loc[:,X_test.columns[:-1]].sum(axis=1)
  return mean_total.mean()

In [ ]:
%%timeit
hitrate_50_faster(X_test)

In [ ]:
hitrate_50_faster(X_test)

0.1622083197197389